# Setting up the TIES protocol with BAC2.0

In this tutorial we will implement the NAMD version of the TIES protocol. We will start with the equilibration steps.

In [1]:
import bac.simulate.namd as nd

In [2]:
md = nd.Run()

We will use the **amber** forcefield, so we need to tell this to the object.

In [3]:
md.amber = True
md.read_exclusions = False
md.parameter_type_CHARMM = False

Now, we set the most important parameters like the timestep and the input files.

In [4]:
md.timestep = 2.0
md.temperature = 300
md.coordinates = '../build/complex.pdb'
md.parameters = '../build/complex.top'
md.binary_output = False
md.output_name = '../equilibration/eq0'

Next, the non bonded interaction setting.

In [5]:
nbc = md.non_bonded_controller
nbc.excluded_interactions = 'scaled1-4'
nbc.one_four_scaling = 0.833333
nbc.cutoff = 12
nbc.switching = True
nbc.switching_distance = 10
nbc.pairlist_distance = 13.5
nbc.non_bonded_frequency = 1
nbc.full_elect_frequency = 2
nbc.steps_per_cycle = 10
nbc.pme = nd.PME(grid_spacing=1.0)

Boundary conditions and constraints:

In [6]:
bc = md.boundary_condition
bc.cell_basis_vector_1 = (87.425, 0.000, 0.000)
bc.cell_basis_vector_2 = (0.000, 97.862, 0.000)
bc.cell_basis_vector_3 = (0.000, 0.000, 76.087)
bc.cell_origin = (-0.071, -0.225, -0.355)
bc.wrap_water = True
bc.wrap_all = True

In [7]:
cc = md.constraints
cc.bond_constraint = nd.BondConstraint(bonds='all', tolerance=0.00001, iterations=100)
cc.harmonic_constraint = nd.HarmonicConstraint(exponent=2, reference_position_file='../build/complex.pdb',
                                               force_constant_file='../constraint/f4.pdb', force_constant_column='B')

In [8]:
md.temperature_controller.langevin = nd.LangevinDynamics(damping=5, temperature=300, hydrogen=False)

Most importantly the free energy calculation parameters:

In [9]:
fe = nd.FreeEnergyController(type='ti', file='../build/tags.pdb', column='B', decouple=True)

fe.output_name = '../equilibration/eq0.alch'
fe.output_frequency = 1000
fe.van_der_waals_shift_coefficient = 5
fe.electronic_interaction_start_lambda = 0.45
fe.van_der_waals_end_lambda = 1.0

md.free_energy_controller = fe

Now the `md` object is ready to run. We need to add it onto an operation queue to actually run in on a supercomputer.

Under the hood, BAC creates an `.conf` file that the NAMD binary knows how to run. You can also have a look at this file by simply encoding the `Run` object. This is how it's done:

In [10]:
from bac.simulate.coding import Encoder, Engine

In [12]:
print(Encoder.encode(md, engine=Engine.namd))

timestep                 2.0
firsttimestep            0
temperature              300.0
coordinates              ../build/complex.pdb
parameters               ../build/complex.top
outputname               ../equilibration/eq0
binaryoutput             no
dcdunitcell              no
amber                    yes
readexclusions           no
scnb                     2
paraTypeXplor            on
paraTypeCharmm           off
gromacs                  False
cutoff                   12.0
switching                on
switchdist               10.0
vdwForceSwitching        off
exclude                  scaled1-4
1-4scaling               0.833333
dielectric               1.0
nonbondedScaling         1.0
vdwGeometricSigma        False
limitdist                0.0
LJcorrection             False
nonbondedFreq            1
fullElectFrequency       2
MTSAlgorithm             impulse/verletI
longSplitting            c1
pairlistdist             13.5
stepspercycle            10
splitPatch               hydrog